In [ ]:
import numpy as np
from numpy.linalg import eig
import matplotlib.pyplot as plt
from matplotlib import cm 

import odesolver as ode

from matplotlib import animation, rc
from IPython.display import HTML

### Numerische Mathematik3 WiSe2023, Prof. Hausser
### Projektarbeit2
# Stiff Differential Equation
## Heehwan Soul, Hajin Suh

### Contents
### 1. Relation between Stiffness and Eigenvalues
    1.1. Engenvalues of A
    1.2. Zeitschrrittbeschaenkung of Various Methods
    1.3. Analytical method for Eigenvalues
### 2. Implementation and Test with Explicit and Implicit Euler Method
    2.1. Case 1: g_1(t) = g_2(t) = 0 and q(x,1) = 1
    2.2. Test with 'Ortsabhängige Anfangsbedingung'
    2.3. Test with 'Orts- und zeitabhängige Quelle'
    2.4. Test with 'Zeitabhängige Dirichlet-Randbedingungen'
### 3. Konvektionsdiffusionsgleichung
    3.1.Derivation of Matrixnotation
    3.2.Solver

--------------------

# 1. Relation between Stiffness and Eigenvalues

In the project, we will discuss about the Parabolic Anfangsrandwertproblem (ARWP) with Linienmethode with finite Difference Method. In the Notebook 9, the equation is derived by 2 steps. Therefore, we will solve the following Anfangswertproblem AWP : 
$$
\boxed{ 
u^\prime (t) = Au + b(t), \, t \in [t_0, t_f], \qquad u(t_0) = u^0
}
$$
$$
u(t) := \begin{pmatrix} u_1(t) \\ u_2(t) \\  \vdots \\ u_{n-2} (t)\\ u_{n-1} (t) \end{pmatrix},  \quad
u^0  := \begin{pmatrix} u^0(x_1) \\ u^0(x_2) \\  \vdots \\ u^0(x_{n-2}) \\ u^0 (x_{n-1}) \end{pmatrix},  \quad
$$
$$
b(t) := \begin{pmatrix} q_1(t) \\ q_2(t) \\ \vdots \\ q_{n-2} \\ q_{n-1} (t) \end{pmatrix} + 
       \frac{D}{h_x^2} \begin{pmatrix} g_1(t) \\ 0 \\ \vdots \\ 0 \\ g_{2} (t) \end{pmatrix}, \quad
A = \frac{D}{h_x^2} 
    \begin{pmatrix} 
              -2 & 1 &  & & \\
              1 & -2 & 1 &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 1& -2  & 1 \\
                & &  & 1 & -2 
              \end{pmatrix}  \in \mathbb{R}^{(n-1)\times (n-1)}
$$

First, we will investigate the Matrix $A$ to decide the stiffness. The statement we wanna prove is that the AWP above gets more stiffer as $n$ increases. For that, we will assume that $D=1$ and $L=1$.

In [ ]:
D = 1.
L = 1.

The following functions create the tridiagonal Matrix $A$ :

In [ ]:
def tridiag(low, mid, up):
    return np.diag(low, -1) + np.diag(mid, 0) + np.diag(up, 1)

def stiffness(m, D, h):
    low = np.ones(m-1)
    mid = -2.*np.ones(m); 
    A  = D/h/h * tridiag(low, mid, low)
    return A

## 1.1. Engenvalues of A

To calculate the matrix $A$, we need the constant mesh grid $h_x = L/n$. The following for-loop is runned for various number of Teilintervall $n$ of the matrix $A$. When the number of Teilintervall is $n$, the dimension of A is $(n-1)  \times  (n-1)$ and also the size of other vectors are $n-1$. In the loop, we calculate $h_x$, corresponding eigenvalues,and the quotient of the maximum and minimum absolute values of eigenvlues. 
$$
\frac{ \max_i |\lambda_i|  } { \min_i | \lambda_i | }$$
The results are visualized.

In [ ]:
# lists to save results
nList = []
hxList = []
evalueList = []
num_evalueList = []
evalueMaxList = []
evalueMinList = []
quotientList = []
for n in range(5, 20):
    nList.append(n)
    
    # constant mesh grid for A
    hx = L/n 
    hxList.append(hx)
    
    # dimension of A = number of Teilintervall - 1
    m = n-1
    # creating tridiagonal matrix for the equation above
    A  = stiffness(m, D, hx)
    
    # eigenvalues
    eigenvalues, _ = eig(A)
    eigenvalues.sort()
    
    # save all eigenvalues for each n
    evalueList.append(eigenvalues)
    
    # how many eigenvalues each n has
    num_evalueList.append(len(eigenvalues))
    
    # Max and min eigenvalues for each n
    evalueMaxList.append(abs(eigenvalues[0]))
    evalueMinList.append(eigenvalues[-1])
    
    # quotient of max and min of absolute of eigenvalues
    quotientList.append(abs(eigenvalues[0])/abs(eigenvalues[-1]))

### Are all eigenvalues real numbers?

In [ ]:
# one example having numpy.float data type
floatType = np.array([4.8])[0]
real_num_evalueList = []

for p in range(len(nList)):
    n = nList[p]
    count = 0
    for q in range(n-1):
        evalue = evalueList[p][q]
        if type(evalue)==type(floatType):
            count +=1
    real_num_evalueList.append(count)

In [ ]:
plt.figure(figsize=(8,4))
plt.rcParams.update({'font.size': 8})

plt.plot(nList, num_evalueList, 'b', label ='all EV')
plt.plot(nList, real_num_evalueList, 'ro', label = 'only real EV')
plt.grid()
plt.xlabel('n')
plt.ylabel('number of (real) eigenvalues')
plt.xticks(np.arange(5,20,1))
plt.yticks(np.arange(4,20,1))
plt.title('number of real eigenvalues depending  on n')
plt.legend()
plt.show()

We can check how many eigenvalues each $n$ has and of them how many are real numbers. The variable 'floatType' below is just one example of numpy.float datatype to compare with all the eigenvalues in the list.
We can see that all the eigenvalues of each $n$ are real numbers. The symmetric matrix has always only real eigenvalues and the matrix $A$ is symmetric as we can see above. Moreover, eigenvectos of symmetric matrix corresponding eigenvectors are orthogonal.

### Are all eigenvalues negative?

In [ ]:
# lists saivng all eigenvalues to draw the graph depending on n    
nGraph = []
evalueGraph = []
for p in range(len(nList)):
    n = nList[p]
    for q in range(n-1):
        nGraph.append(n)
        evalueGraph.append(evalueList[p][q])

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams.update({'font.size': 8})

plt.subplot(211)
plt.scatter(nGraph, evalueGraph, s=8, color = 'b')
plt.xlabel('n')
plt.ylabel('eigenvalues')
plt.title('all eigenvalues depending  on n')

plt.subplot(212)
plt.plot(nList, evalueMinList, 'bo')
plt.title('minimum absolute of eigenvalues depending  on n')
plt.show()

From the second graph, we can see the minimum absolute value of eigenvalues for each $n$ is negative. Also from the first graph dots for the same $n$ mean all the eigenvalues for that $n$. We can see all eigenvalues for each $n$ are all negative. When the matrix $A$ is negative definitiv, its eigenvalues are all negative. When all eigenvalues are negative, the process has stability which means $u$ approach $0$ when $t$ gets bigger. In the scalar cases, we could already learn that the function gets smaller to $0$ when eigenvalue is negative.

### Quotient of max and min of absolute of eigenvalues?

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams.update({'font.size': 8})

plt.subplot(211)
plt.plot(nList, quotientList,'bo')
plt.grid()
plt.xlabel('n')
plt.ylabel('Quotienten')
plt.title('Quotienten von eigenwerten Max/Min')


plt.subplot(212)
plt.plot(nList, quotientList,'bo', label='quotient')
t = np.linspace(5,20,100)
plt.plot(t, t**2, label='y=t^2')
plt.xlabel('n')
plt.ylabel('Quotienten')
plt.title('Quotienten with Log scale')
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.show()

As $n$ gets bigger, the quotient of max and min of eigenvalues gets bigger. When one element of the solution vector $u(t)$ is way more rapidly changing compared to other elements, the AWP can be called as stiff equation. 

When all eigenvalues are negative like the matrix $A$, the values of each elements of $u(t)$ decrease. When the absolute value of one of eigenvalues gets bigger, the Zeitschrittebeschranke gets smaller. And with this small Zeitschrittebeschrake, other elements decrease very slow. The size of Zeitschritt for numerical solver is decided by the element with the maximum absolute of eigenvalue but the time to be solved is decided by the element decreasing the most slowly. 

If eigenvalues are similary big or similary small together, it is not a problem, but if one absolute value is very big compared to other values, other elements approach 0 too slowly. So we can say that when quotient of maximum and minimum of absolute of eigenvalues gets bigger, the equation is stiffer. We can assume it gets stiffer as n gets bigger.

## 1.2. Zeitschrrittbeschaenkung of Various Methods


So that differential equation has an asymptotic stability to be solved, the Zeitschrittweit $\tau$ for the numberical solvers should be limited as :

For explicit Euler method :
$$\boxed{ \tau < \frac{2}{| \lambda_\mathrm{max} |} } $$
For Runge-Kutta method :
$$\boxed{ \tau < \frac{2 \sqrt{2}}{| \lambda_\mathrm{max} |} } $$

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams.update({'font.size': 8})

plt.subplot(211)
tauEulerList = 2/np.array(evalueMaxList)
tauRKVList = 2*np.sqrt(2)/np.array(evalueMaxList)
plt.plot(nList, tauEulerList, 'r', label='explizit euler')
plt.plot(nList, tauRKVList, 'b', label='RKV')
plt.grid()
plt.xlabel('n')
plt.ylabel('tau_max')
plt.title('Zeitschrittbeschranke')
plt.legend()

plt.subplot(212)
n_exp_List = []
evalue_exp_List = []
evalueMax_exp_List = []

for i in range(2,12):
    n = 2**i
    n_exp_List.append(n)
    
    hx = L/n 
    m = n-1
    A  = stiffness(m, D, hx)  
    eigenvalues, _ = eig(A)
    eigenvalues.sort()
    evalue_exp_List.append(eigenvalues)
    evalueMax_exp_List.append(abs(eigenvalues[0]))
    
tauEulerList = 2/np.array(evalueMax_exp_List)
tauRKVList = 2*np.sqrt(2)/np.array(evalueMax_exp_List)
plt.plot(n_exp_List, tauEulerList, 'r', label='explizit euler')
plt.plot(n_exp_List, tauRKVList, 'b', label='RKV')
t = np.linspace(5,2**11,1000)
plt.plot(t, t**(-2), 'g', label='y=n^(-2)')
plt.xlabel('n')
plt.ylabel('tau_max')
plt.title('Zeitschrittbeschranke (Log scale)')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

From the first graph, as $n$ gets bigger, the Zeitschrittbeschranke($\tau_{max}$) gets smaller. Small $\tau_{max}$ means the size of Zeitschritt $\tau$ must be small to make the equation having asymptotic stability. Then we can say it is stiff. Same as the result of 1.1., as $n$ gets bigger, the size of Zeitschritt $\tau$ needs to be smaller and the eqaution is more stiffer. 

Moreover $\tau_{max}$ of Runge-Kutta method is bigger than $\tau_{max}$ of explicit Euler method. So we can assume that Runge-Kutta method is better than explicit Euler method to solve the stiff equation.

From the second graph, the graphs with log scale look like linear functions. They are parallel with $y=n^{-2}$. It means the value of Zeitschrittebeschranke is inversly proportional to $n^{2}$.

To understand the reason analytically, we can use the formula that will be addressed in 1.3. 
$$ \lambda = \frac{-4D}{h_x^2} sin^{2}(\frac{j\pi}{2n}) $$
We will investigate using it for explicit Euler method.
$$\tau < \frac{2}{| \lambda_\mathrm{max} |}$$
$$ = \frac{2}{max (| \frac{-4D}{h_x^2} sin^{2}(\frac{j\pi}{2n})|)} $$
Since $D=1$,
$$ = \frac{h_x^2}{max (2sin^{2}(\frac{j\pi}{2n}))} $$
Since $L=1$ and $h_{x}= \frac{L}{n} = \frac{1}{n}$,
$$ = \frac{(\frac{1}{n})^2}{max (2sin^{2}(\frac{j\pi}{2n}))} $$
Since $ 0 \leq  sin^{2}(\frac{j\pi}{2n}) \leq   1$, 
$$max (2sin^{2}(\frac{j\pi}{2n})) = 2$$
Therefore, for explicit euler,
$$\tau_{max} = \frac{1}{2}n^{-2}$$

If we draw $ y = \frac{1}{2}n^{-2}$ on the same axis with graph for explicit euler method,

In [ ]:
tauEulerList = 2/np.array(evalueMax_exp_List)
plt.plot(n_exp_List, tauEulerList, 'ro', label='explizit euler')
t = np.linspace(5,2**11,1000)
plt.plot(t, 0.5*t**(-2), 'g', label='y=0.5*n^(-2)')
plt.xlabel('n')
plt.ylabel('tau_max')
plt.title('Zeitschrittbeschranke of explicit euler')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

## 1.3. Analytical method for Eigenvalues

Even though in the most of general case, the eigenvalues cannot be calculated analytically, the given equation has an analytic formula of eigenvalues. 
$$
\lambda_j = -\frac{4 D}{h_x^2} \sin^2 ( j \pi / 2n ), \quad j=1,2, \dots, n-1
$$
First we will derive this formula with Chebyshev Polynomial and Eigenfunction $ v_\omega(t) = \sin \omega t$. Followed by the formunal will be verified with matrix $A$ wiht $n=5$

### Formula Derivation with Chebyshev Polynomial

$$
A = \frac{D}{h_x^2} 
    \begin{pmatrix} 
              -2 & 1 &  & & \\
              1 & -2 & 1 &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 1& -2  & 1 \\
                & &  & 1 & -2 
              \end{pmatrix}  \in \mathbb{R}^{(n-1)\times (n-1)}
$$

To find eigenvalues, we should solve $det(A- \lambda I )=0$
$$
A- \lambda I = \frac{D}{h_x^2} 
    \begin{pmatrix} 
              -2 & 1 &  & & \\
              1 & -2 & 1 &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 1& -2  & 1 \\
                & &  & 1 & -2 
              \end{pmatrix} - \begin{pmatrix} 
              \lambda & 0 &  & & \\
              0 & \lambda & 0 &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 0& \lambda  & 0 \\
                & &  & 0 & \lambda 
              \end{pmatrix}
$$
$$
A- \lambda I = \frac{D}{h_x^2} 
    \begin{pmatrix} 
              -2 & 1 &  & & \\
              1 & -2 & 1 &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 1& -2  & 1 \\
                & &  & 1 & -2 
              \end{pmatrix} - 
              \frac{D}{h_x^2}
              \begin{pmatrix} 
              \lambda\frac{h_x^2}{D} & 0 &  & & \\
              0 & \lambda\frac{h_x^2}{D} & 0 &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 0& \lambda\frac{h_x^2}{D}  & 0 \\
                & &  & 0 & \lambda\frac{h_x^2}{D} 
              \end{pmatrix}
$$
$$
A- \lambda I = \frac{D}{h_x^2} 
    \begin{pmatrix} 
              -2-\lambda\frac{h_x^2}{D} & 1 &  & & \\
              1 & -2-\lambda\frac{h_x^2}{D} & 1 &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 1& -2-\lambda\frac{h_x^2}{D}  & 1 \\
                & &  & 1 & -2-\lambda\frac{h_x^2}{D} 
              \end{pmatrix}
$$

$A- \lambda I$ is tridiagonal Matrix. 

Determinant of tridiagonal Matrix is : 
$$M = \begin{pmatrix} 
              a & b &  & & \\
              c & a & b &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & c& a  & b \\
                & &  & c & a 
              \end{pmatrix} \in \mathbb{R}^{(n)\times (n)}$$
If $D_{n}$ is determinant when dimension of $M$ is $n$, it can be expressed as a recursive equation.
$$D_{n} = a \cdot D_{n-1}-bc\cdot D_{n-2}$$
When $a$ and $b$ are $1$, 
$$D_{n} = a \cdot D_{n-1}-D_{n-2}$$

The shape of this recursive is close to Chebyshev polynomial of the second kind. The recursive formula of 2.Chebyshev polynomial is $$U_{n}(x) = 2x \cdot U_{n-1}(x)-U_{n-2}(x)$$
If we rewrite as $$D_{n} = a \cdot D_{n-1}-D_{n-2} = 2 \frac{a}{2} \cdot D_{n-1}-D_{n-2}$$
the determinant can be expressed as $U_{n}(\frac{a}{2})$

The definition of 2.Chebyshev polynomial is 
$$ U_{n}(x) = \frac{1}{n+1}T_{n+1}'(x) $$
Here $T_{n}(x)$ is 1.Chebyshev polynomial and the definition is $$T_{n}(x)= cos(ncos^{-1}x)$$
Therefore,
$$ U_{n}(x) = \frac{1}{n+1}(cos((n+1)cos^{-1}x))'$$
$$ = \frac{1}{n+1} sin((n+1)cos^{-1}x) (n+1)\frac{1}{\sqrt{1-x^{2}}}$$
$$ = sin((n+1)cos^{-1}x)\frac{1}{\sqrt{1-x^{2}}}$$

Let $cos^{-1}x = \theta $, $cos\theta=x$ and $sin\theta=\sqrt{1-x^{2}}$.
$$ U_{n}(x) = \frac{sin((n+1)\theta)}{sin\theta}$$
$$ U_{n}(cos\theta) = \frac{sin((n+1)\theta)}{sin\theta}$$

Roots of $ U_{n}(x) = 0$ are : 
$$ U_{n}(x) = U_{n}(cos\theta) = \frac{sin((n+1)\theta)}{sin\theta}=0$$
$$ sin((n+1)\theta) = 0 $$
$$ (n+1)\theta = j\pi$$
$$\theta = \frac{j\pi}{n+1}  $$
$$\theta = cos^{-1}x = \frac{j\pi}{n+1}  $$
$$ x = cos(\frac{j\pi}{n+1})  $$

Then for $A- \lambda I_{n-1}$, $a= -2-\lambda\frac{h_x^2}{D}$,
$$ det(A- \lambda I_{n-1}) = \frac{D}{h_x^2} U_{n-1}(\frac{a}{2})$$
$$= \frac{D}{h_x^2} U_{n}(\frac{ -2-\lambda\frac{h_x^2}{D}}{2}) = -\frac{D}{h_x^2} U_{n}(\frac{ 2+\lambda\frac{h_x^2}{D}}{2}) $$
Since $j = 1,2,...,n-1$
$$ \frac{ 2+\lambda\frac{h_x^2}{D}}{2} = cos(\frac{j\pi}{(n-1)+1}) $$
$$ 2+\lambda\frac{h_x^2}{D} = 2cos(\frac{j\pi}{n})$$
$$ \lambda\frac{h_x^2}{D} = 2cos(\frac{j\pi}{n}) - 2$$
$$ \lambda = 2(cos(\frac{j\pi}{n}) - 1) \cdot \frac{D}{h_x^2} $$
$$ \lambda = 2(cos(2\cot \frac{j\pi}{2n}) - 1) \cdot \frac{D}{h_x^2} $$
Using double angle formula,
$$ \lambda = 2(1-2sin^{2}(\frac{j\pi}{2n}) - 1) \cdot \frac{D}{h_x^2} $$
$$ \lambda = 2(-2sin^{2}(\frac{j\pi}{2n})) \cdot \frac{D}{h_x^2} $$
Therefore for $j = 1,2,...,n-1$
$$ \lambda_j = \frac{-4D}{h_x^2} sin^{2}(\frac{j\pi}{2n}) $$

### Formula Derivation with Eigenfunction $ v_\omega(t) = \sin \omega t$ 

$$ Av = \lambda v$$
$$ A \begin{pmatrix} sin(\frac{j\pi}{n}) 
\\ sin(2\frac{j\pi}{n}) \\ \vdots 
\\ sin((n-2) \frac{j\pi}{n}) \\ sin((n-1)\frac{j\pi}{n}) \end{pmatrix} = \lambda_{j} \begin{pmatrix} sin( \frac{j\pi}{n}) 
\\ sin(2\frac{j\pi}{n}) \\ \vdots 
\\ sin((n-2) \frac{j\pi}{n}) \\ sin((n-1) \frac{j\pi}{n}) \end{pmatrix}, j = 1,2,..., n-1$$

$$
\frac{D}{h_x^2} 
    \begin{pmatrix} 
              -2 & 1 &  & & \\
              1 & -2 & 1 &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 1& -2  & 1 \\
                & &  & 1 & -2 
              \end{pmatrix} 
\begin{pmatrix} sin(\frac{j\pi}{n}) 
\\ sin(2\frac{j\pi}{n}) \\ \vdots 
\\ sin((n-2) \frac{j\pi}{n}) \\ sin((n-1)\frac{j\pi}{n}) \end{pmatrix} = \lambda_{j} \begin{pmatrix} sin( \frac{j\pi}{n}) 
\\ sin(2\frac{j\pi}{n}) \\ \vdots 
\\ sin((n-2) \frac{j\pi}{n}) \\ sin((n-1) \frac{j\pi}{n}) \end{pmatrix}$$

If we see $\omega$.th row,
$$ \frac{D}{h_x^2} ( 1\cdot sin((\omega-1) \frac{j\pi}{n})
-2\cdot  sin(\omega \frac{j\pi}{n}) 
+ 1\cdot sin((\omega+1) \frac{j\pi}{n})) 
= \lambda_{j} sin(\omega \frac{j\pi}{n})$$

Expanding the sum of angles : 
$$ \frac{D}{h_x^2} 
(sin(\omega\frac{j\pi}{n})cos(\frac{j\pi}{n})-cos(\omega\frac{j\pi}{n})sin(\frac{j\pi}{n})
-2sin(\omega \frac{j\pi}{n}) 
+ (sin(\omega\frac{j\pi}{n})cos(\frac{j\pi}{n})+cos(\omega\frac{j\pi}{n})sin(\frac{j\pi}{n})
= \lambda_{j} sin(\omega \frac{j\pi}{n})$$

Since $cos(\omega\frac{j\pi}{n})sin(\frac{j\pi}{n})$ is deleted,
$$ \frac{D}{h_x^2} 
(sin(\omega\frac{j\pi}{n})cos(\frac{j\pi}{n})
-2sin(\omega \frac{j\pi}{n}) 
+ sin(\omega\frac{j\pi}{n})cos(\frac{j\pi}{n}) )
= \lambda_{j} sin(\omega \frac{j\pi}{n})$$

$$ \frac{D}{h_x^2} sin(\omega\frac{j\pi}{n}) (
2cos(\frac{j\pi}{n})-2 )
= \lambda sin(\omega \frac{j\pi}{n})$$

$$ 2\frac{D}{h_x^2}(cos(\frac{j\pi}{n})-1 ) = \lambda_{j} $$
$$ \lambda_{j} = \frac{2D}{h_x^2}(cos(2\cot \frac{j\pi}{2n}) - 1) $$
Using double angle formula,
$$ \lambda_{j} =  \frac{2D}{h_x^2}(1-2sin^{2}(\frac{j\pi}{2n}) - 1) $$
$$ \lambda_{j} = \frac{2D}{h_x^2}(-2sin^{2}(\frac{j\pi}{2n})) $$
Therefore for $j = 1,2,...,n-1$
$$ \lambda_{j} = \frac{-4D}{h_x^2} sin^{2}(\frac{j\pi}{2n}) $$

### Verifying Analytical value of Eigenvalues

For simple verifying, we will set the value $n = 5$. To compare the eigenvector calculated analytically, we changed $v_{4}$ of eigenvector $v$ to $1$. Firts code is finding eigenvalues and eigenvectors using inner function "np.linalg.eig". Second code is finding eigenvalues with formula. To find eigenvector from them, we need the function for gaus elimination.

In [ ]:
def gaussElimin(a):
    n = len(a)
    for k in range(0, n-1):
        for i in range(k+1, n):
            if a[i, k] != 0.0:
                lam = a[i, k] / a[k, k]
                a[i, k:n] = a[i, k:n] - lam*a[k, k:n]
    return a

In [ ]:
# np.linalg.eig
n=5
h = L/n
m = n-1
A  = stiffness(m, D, h)  
eigenvalues, eigenvectors = eig(A)

# v4 to 1
eigenvectors = eigenvectors/[eigenvectors[3][0],eigenvectors[3][1],eigenvectors[3][2],eigenvectors[3][3]]

eigenDic = {}
for k in range(4):
    eigenvalue = eigenvalues[k]
    eigenvector = np.array([eigenvectors[0][k], eigenvectors[1][k],
                               eigenvectors[2][k], eigenvectors[3][k]])
    eigenDic[eigenvalue] = eigenvector
    print( k+1, ".eigenvalue is ", eigenvalue)
    print("its corresponding eigenvector is ", eigenvector, "\n")

In [ ]:
# with formula
eigen_anaDic = {}
for j in range(1,n):
    eigenvalue = -4*D/h/h * (np.sin(j*np.pi/(2*n)))**2
    I = np.identity(n-1)
    M = A-eigenvalue*I
    a = gaussElimin(M)
    x4 = 1
    x3 = -x4*a[2][3]/a[2][2]
    x2 = -x3*a[1][2]/a[1][1]
    x1 = -x2*a[0][1]/a[0][0]
    eigenvector = np.array([x1,x2,x3,x4])
    
    eigen_anaDic[eigenvalue] = eigenvector
    print( j, ".eigenvalue by formula is ", eigenvalue)
    print("its corresponding eigenvector is ", eigenvector, "\n")

If we compare the first result by inner function  "np.linalg.eig" and the second result solved analytically, we can verify that their result is identical.

# 2. Implementation and test with explicit and implicit euler methods

We are going to use explicit and implicit euler methods for the tests.

## 2.1. Case 1: $g_1(t) = g_2(t) = 0$ and $q(x,1) = 1$

In [ ]:
def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

### Implicit Method

In [ ]:
import numpy as np
import odesolver as ode
import matplotlib.pyplot as plt
from matplotlib import cm   # colormaps
#%matplotlib widget

n = 20
D = 1
L = 1

#A
diagonal = -2*np.ones(n-1)
over = np.ones(n-2)
matrix = tridiag(over, diagonal, over)
hx = L/n
A = (D/hx**2)*matrix

#for the case of Quelle q(x, 1) = 1 and g_1(t) = g_2(t) = 0
#b
def g1(t):
    return 0

def g2(t):
    return 0

# first part of b
def q(j, t): # Quelle = 1
    return 1

def b(t):
    b_1 = np.zeros(n-1)
    
    for i in range (n-1):
        b_1[i] = q(i+1, t)

    # second part of b(vector)    
    b_2 = np.zeros(n-1)
    b_2[0] = g1(t)
    b_2[-1] = g2(t)
    
    return b_1 + (D/hx**2)*b_2

#f
def f(t, y): # y means u here
    return np.matmul(A,y) + b(t)

def f_jac(t, y):
    return A


# Anfangsbedingung
u0 = np.zeros(n-1) # from u_1 to u_(n-1)
t0 = 0
tf = 0.8
t_span = (t0, tf)       # Zeitintervall als Tupel

N  = 400   # Azahl Zeitschritte
tk = np.linspace(t0, tf, N+1)

xj = np.linspace(0, L, n+1)
y_all_im = np.zeros( (n+1, N+1) )  # erster Index Ort, zweiter Index Zeit

solution_im = ode.euler_backward(f, t_span, u0, tk, f_jac)
u_k_im = solution_im.y # from u_1 to u_(n-1)

# add the 'Randpunkte'
y_all_im[0, :] = g1(tk)
y_all_im[-1, :] = g2(tk)

# add the points, which are produced by the implicit euler method 
for k in range(1,n):
    
    y_all_im[k, :] = u_k_im[k-1, :]

TK, XJ = np.meshgrid(tk, xj)    # erzeugt Matrizen derselben Dimension wir y_all


fig = plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')
ax.plot_wireframe(TK, XJ, y_all_im)
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_zlabel('u')

### Explicit Method

In [ ]:
y_all_ex = np.zeros( (n+1, N+1) )  # erster Index Ort, zweiter Index Zeit

solution_ex = ode.euler_forward(f, t_span, u0, tk)
u_k_ex = solution_ex.y # from u_1 to u_(n-1)

# add the 'Randpunkte'
y_all_im[0, :] = g1(tk)
y_all_im[-1, :] = g2(tk)

# add the points, which are produced by the explicit euler method
for k in range(1,n):
    
    y_all_ex[k, :] = u_k_ex[k-1, :]

TK, XJ = np.meshgrid(tk, xj)    # erzeugt Matrizen derselben Dimension wir y_all

fig = plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')
ax.plot_wireframe(TK, XJ, y_all_ex)
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_zlabel('u')

### Compare the results between using the implicit euler method and the explicit euler method

Let's see the result with implicit euler method. The form of the graph in terms of x(t is fixed) is the form of quadratic function with the negative sign of the coeffcient of $x^2$. But when we see the result with explicit euler method, the values are going up and down dramatically after a cetain time and there is no form of quadratic function. This is because it is a stiff problem and the method of explicit euler method is not stable. On the other hand, the implicit euler method is stable, so that we could get a good result.

### Why the form of quadratic function?

We can see the datails at the end of 2.2.

### Visualization of the result with Implicit euler method as Film 

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

#%matplotlib qt
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ymax = np.max(y_all_im)
ymin = np.min(y_all_im)
ax.set_xlim(( 0, L))
ax.set_ylim((ymin, ymax))

line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(i):
    y = y_all_im[:,i]
    line.set_data(xj, y)
    return (line,)


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(tk), interval=50, blit=True) # intervall in milli seconds

display(HTML(anim.to_jshtml()))
plt.show()

#HTML(anim.to_html5_video())

## 2.2. Test with 'Ortsabhängige Anfangsbedingung'

We are going to see what happen when $u_k(0)=0.06$ instead of $u_k(0)=0.00$ (k is from 1 to n-1). The other conditions are as same as the Case 1.

### Visualization as Surface

In [ ]:
# Anfangsbedingung
u0 = 0.06*np.ones(n-1) # from u_1 to u_(n-1)

solution_im = ode.euler_backward(f, t_span, u0, tk, f_jac)
u_k_im = solution_im.y # from u_1 to u_(n-1)

y_all_im = np.zeros( (n+1, N+1) )  # erster Index Ort, zweiter Index Zeit

# add the 'Randpunkte'
y_all_im[0, :] = g1(tk)
y_all_im[-1, :] = g2(tk)

# add the points, which are produced by the implicit euler method 
for k in range(1,n):
    
    y_all_im[k, :] = u_k_im[k-1, :]

TK, XJ = np.meshgrid(tk, xj)    # erzeugt Matrizen derselben Dimension wir y_all


fig = plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')
ax.plot_wireframe(TK, XJ, y_all_im)
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_zlabel('u')


### Visualization as Film

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

#%matplotlib qt
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ymax = np.max(y_all_im)
ymin = np.min(y_all_im)
ax.set_xlim(( 0, L))
ax.set_ylim((ymin, ymax))

line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(i):
    y = y_all_im[:,i]
    line.set_data(xj, y)
    return (line,)


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(tk), interval=50, blit=True) # intervall in milli seconds

display(HTML(anim.to_jshtml()))
plt.show()

#HTML(anim.to_html5_video())

When t is big enough, the graph of $u_k(0)=0.06$ is as same as the graph of $u_k(0)=0.00$(Here k is from 1 to n-1). The reason is as follows:

When $g_1(t) = g_2(t) = 0$, we can write the AWP like this

$$
	 \partial_{t}u = \partial_{xx}u + q.
$$

When q is 1 and t is big enough we can write it as

$$
0 = u^{\prime\prime}(x) + 1, \quad u(0) = u(L=1) = 0.
$$

Then
$$
u(x) = \frac{1}{2}x^2 + C_1x + C_2, \quad u(0) = u(L=1) = 0.
$$

Finally
$$
u(x) = \frac{1}{2}x(x-L), \quad L=1
$$

We can see that the same u(x) is calculated when we use the same 'Randbedingungen($g_1(t) = g_2(t) = 0$)'. That is why these two graphes are same when t is big enough.

## 2.3. Orts- und zeitabhängige Quelle

We are going to change the defintion of the function q(j, t) for the test of 'Orts- und zeitabhängige Quelle'. Other conditions are as same as our Case 1(2.1).

### Visualization as Surface

In [ ]:
# Anfangsbedingungen
u0 = np.zeros(n-1) # from u_1 to u_(n-1)

y_all_im = np.zeros( (n+1, N+1) )  # erster Index Ort, zweiter Index Zeit

def q(j, t): # Quelle
    return xj[j] + t
  
def b(t):
    b_1 = np.zeros(n-1)
    
    for i in range (n-1):
        b_1[i] = q(i+1, t)

    b_2 = np.zeros(n-1)
    b_2[0] = g1(t)
    b_2[-1] = g2(t)
    
    return b_1 + (D/hx**2)*b_2

def f(t, y): # y means u here
    return np.matmul(A,y) + b(t)

solution_im = ode.euler_backward(f, t_span, u0, tk, f_jac)
u_k_im = solution_im.y # from u_1 to u_(n-1)

# add the 'Randpunkte'
y_all_im[0, :] = g1(tk)
y_all_im[-1, :] = g2(tk)

# add the points, which are produced by the implicit euler method 
for k in range(1,n):
    
    y_all_im[k, :] = u_k_im[k-1, :]

TK, XJ = np.meshgrid(tk, xj)    # erzeugt Matrizen derselben Dimension wir y_all


fig = plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')
ax.plot_wireframe(TK, XJ, y_all_im)
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_zlabel('u')

### Visualization as Film

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

#%matplotlib qt
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ymax = np.max(y_all_im)
ymin = np.min(y_all_im)
ax.set_xlim(( 0, L))
ax.set_ylim((ymin, ymax))

line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(i):
    y = y_all_im[:,i]
    line.set_data(xj, y)
    return (line,)


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(tk), interval=50, blit=True) # intervall in milli seconds

display(HTML(anim.to_jshtml()))
plt.show()

#HTML(anim.to_html5_video())

We can find out that the slopes change slower than the case 1, becasuse we have different 'Quelle' from the case 1(q = 1). It lead to the different shape of graph.

## 2.4. Test with 'Zeitabhängige Dirichlet-Randbedingungen'

We are going to change the defintions of $g_1(t)$ and $g_2(t)$ for the test of 'Zeitabhängige Dirichlet-Randbedingungen'. Other conditions are as same as our Case 1(2.1).

### Visualization as Surface

In [ ]:
# Anfangsbedingungen
u0 = np.zeros(n-1) # from u_1 to u_(n-1)

def g1(t):
    return 10*(1+np.sin(t))

def g2(t):
    return 10

# first part of b
def q(j, t): # Quelle q = 1 as our Case 1
    return 1
  
def b(t):
    b_1 = np.zeros(n-1)
    
    for i in range (n-1):
        b_1[i] = q(i+1, t)

    b_2 = np.zeros(n-1)
    b_2[0] = g1(t)
    b_2[-1] = g2(t)
    
    return b_1 + (D/hx**2)*b_2

def f(t, y): # y means u here
    return np.matmul(A,y) + b(t)

solution_im = ode.euler_backward(f, t_span, u0, tk, f_jac)
u_k_im = solution_im.y # from u_1 to u_(n-1)

y_all_im = np.zeros( (n+1, N+1) )  # erster Index Ort, zweiter Index Zeit

# add the 'Randpunkte'
y_all_im[0, :] = g1(tk)
y_all_im[-1, :] = g2(tk)


# add the points, which are produced by the implicit euler method 
for k in range(1,n):
    
    y_all_im[k, :] = u_k_im[k-1, :]

TK, XJ = np.meshgrid(tk, xj)    # erzeugt Matrizen derselben Dimension wir y_all


fig = plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')
ax.plot_wireframe(TK, XJ, y_all_im)
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_zlabel('u')

### Visualization as Film

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

#%matplotlib qt
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ymax = np.max(y_all_im)
ymin = np.min(y_all_im)
ax.set_xlim(( 0, L))
ax.set_ylim((ymin, ymax))

line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(i):
    y = y_all_im[:,i]
    line.set_data(xj, y)
    return (line,)


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(tk), interval=50, blit=True) # intervall in milli seconds

display(HTML(anim.to_jshtml()))
plt.show()

#HTML(anim.to_html5_video())

When we compare this result with the result of Case 1(2.1), the graphs are very different. We can see the values of $u_0(t)$ is getting bigger from 10 to $10(1+sin(0.8))$ and the values of $u_n(t)$ is always 10.

# 3. Konvektionsdiffusionsgleichung

## 3.1.Derivation of Matrixnotation

First we have learned the Kontinuitätsgleichung
$$\partial_t u(x,t)  =  - \partial_x j(x,t) + q(x, t)$$

Depending on system, $j$ and $q$ can be decided.

For the Konstitutive Gleichung such as Diffusionsgleichung, 
$$ j (x,t) = j_\mathrm{diff} (x,t) = - D \partial_x u(x,t)  $$

For the Konvektionsdiffusionsgleichung, 
$$ j (x,t) = j_\mathrm{diff} (x,t) + j_\mathrm{konv} (x,t) $$
$$ j_\mathrm{konv} (x,t) = v u(x,t) \qquad  $$
, where $v$ is velocity field.

Therefore,
$$ j (x,t) = - D \partial_x u(x,t) + v u(x,t) $$ 

If we substitute $ j (x,t) $ to the Kontinuitätsgleichung,
$$\partial_t u(x,t)  =  - \partial_x (- D \partial_x u(x,t) + v u(x,t)) + q(x, t)$$
$$  =  D \partial_{xx} u(x,t) - \partial_x v u(x,t) + q(x, t)$$


Aus Finite Differenzen Methode,
$$	 \partial_{xx}u(x_j, t)  \approx   \frac{ u(x_{j+1}, t) - 2u(x_j, t) + u(x_{j-1}, t) }{h_x^2} $$

Aus Zentralen Differenzenquotienten
$$	 \partial_{x}u(x_j, t)  \approx   \frac{ u(x_{j+1}, t) -  u(x_{j-1}, t) }{2h_x} $$

If we substitute $\partial_{xx}u(x_j, t)$ und $\partial_{x}u(x_j, t)$ to the equation of $\partial_t u(x,t)$ above,
$$\partial_t u(x_j,t)  =  D \frac{ u(x_{j+1}, t) - 2u(x_j, t) + u(x_{j-1}, t) }{h_x^2} 
                        - v \frac{ u(x_{j+1}, t) -  u(x_{j-1}, t) }{2h_x} + q(x_j, t)$$
$$ = \frac{D}{h_x^2}u_{j+1}-2\frac{D}{h_x^2}u_{j}+ \frac{D}{h_x^2}u_{j-1}  - \frac{v}{2h_x}u_{j+1} + \frac{v}{2h_x}u_{j-1} + q_j$$

$$ = u_{j-1}( \frac{D}{h_x^2} +  \frac{v}{2h_x}) - \frac{2D}{h_x^2}u_{j} + u_{j+1} (\frac{D}{h_x^2} - \frac{v}{2h_x} ) + q_j$$

Therefore the differential equation of the Konvektionsdiffusionsgleichung is :
$$\partial_t u(x_j,t)  = \frac{1}{2h_x^2} 
\left [ u_{j-1}(2D +vh_x) - 4D \cdot u_{j} + u_{j+1} (2D  -vh_x) \right ] 
+ q_j$$

With Dirichlet-boundary condition, 
$$u_0(t) = g_1(t), \quad u_n(t) = g_n(t) $$
The initial Dichte is  $u^0(x)$,
    $$ u_j(t_0) = u^0(x_j), \qquad j = 1, \dots, n-1 $$

**Matrixnotation:** 
To change this equation to Matrixnotation, we should check the boundary condition which means when $u(x_0,t) = u_0(t) = g_1(t)$ and $u(x_n,t) = u_n(t) = g_2(t)$.

Let
$$u(t) := \begin{pmatrix} u_1(t) \\ u_2(t) \\  \vdots \\ u_{n-2} (t)\\ u_{n-1} (t) \end{pmatrix} $$

We can firstly assume the matrix notation without considerinig the boundary condition. For $j=1,2,....n-1$
$$ \begin{pmatrix} \partial_t u_1(t) \\ \partial_t u_2(t) \\  \vdots \\ \partial_t u_{n-2} (t)\\ \partial_t u_{n-1} (t) \end{pmatrix} = 
\frac{1}{2h_x^2} 
    \begin{pmatrix} 
              -4D & 2D-vh_x &  & & \\
              2D+vh_x & -4D & 2D-vh_x &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 2D+vh_x& -4D  & 2D-vh_x \\
                & &  & 2D+vh_x & -4D 
              \end{pmatrix}
\begin{pmatrix} u_1(t) \\ u_2(t) \\  \vdots \\ u_{n-2} (t)\\ u_{n-1} (t) \end{pmatrix}
+ \begin{pmatrix} q_1(t) \\ q_2(t) \\ \vdots \\ q_{n-2} \\ q_{n-1} (t) \end{pmatrix} + \Phi $$

Comparing the equation and matrix notation for $j=1$ and $j=n-1$, we have to find $\Phi$.
- When $j=1$, the equation that we need to solve is 
$$\partial_t u_1  = \frac{1}{2h_x^2}  \left [ u_{0}(2D +vh_x) - 4D \cdot u_{1} + u_{2} (2D  -vh_x) \right ] + q_1$$
However the equation we can get from the matrixnotation above is
$$\partial_t u_1  = \frac{1}{2h_x^2}  \left [- 4 \cdot Du_{1} + u_{2} (2D  -vh_x) \right ] + q_1$$
This matrixnotation needs $\frac{1}{2h_x^2} u_{0}(2D +vh_x)$ to be identical with equation.


- When $j=n-1$, the equation that we need to solve is 
$$\partial_t u_{n-1}  = \frac{1}{2h_x^2}  \left [ u_{n-2}(2D +vh_x) - 4D \cdot u_{n-1} + u_{n} (2D  -vh_x) \right ] + q_1$$
However the equation we can get from the matrixnotation above is
$$\partial_t u_{n-1}  = \frac{1}{2h_x^2}  \left [ u_{n-2}(2D +vh_x) - 4 \cdot Du_{n-1} \right ] + q_1$$
This matrixnotation needs $\frac{1}{2h_x^2} u_{n} (2D  -vh_x)$ to be identical with equation.

Therefore, we can set $\Phi$ as :
$$ \Phi = \frac{1}{2h_x^2}\begin{pmatrix} u_{0}(2D +vh_x) \\ 0\\  \vdots \\ 0\\u_{n} (2D  -vh_x) \end{pmatrix}
 = \frac{1}{2h_x^2}\begin{pmatrix} g_{1}(2D +vh_x) \\ 0\\  \vdots \\ 0\\g_{2} (2D  -vh_x) \end{pmatrix} \in \mathbb{R}^{(n-1)} $$

Now we can make the complete matrixnotation for the Konvektionsdiffusionsgleichung:

$$
u(t) := \begin{pmatrix} u_1(t) \\ u_2(t) \\  \vdots \\ u_{n-2} (t)\\ u_{n-1} (t) \end{pmatrix},  \quad
u^0  := \begin{pmatrix} u^0(x_1) \\ u^0(x_2) \\  \vdots \\ u^0(x_{n-2}) \\ u^0 (x_{n-1}) \end{pmatrix},  \quad$$

$$ b(t) := \begin{pmatrix} q_1(t) \\ q_2(t) \\ \vdots \\ q_{n-2} \\ q_{n-1} (t) \end{pmatrix} + 
    \frac{1}{2h_x^2}\begin{pmatrix} g_{1}(t)(2D +vh_x) \\ 0\\  \vdots \\ 0\\g_{2}(t) (2D  -vh_x) \end{pmatrix} $$
    
$$  A = \frac{1}{2h_x^2} 
    \begin{pmatrix} 
              -4D & 2D-vh_x &  & & \\
              2D+vh_x & -4D & 2D-vh_x &  & \\
                & \ddots & \ddots & \ddots & \\
                &  & 2D+vh_x& -4D  & 2D-vh_x \\
                & &  & 2D+vh_x & -4D 
              \end{pmatrix}  \in \mathbb{R}^{(n-1)\times (n-1)}
$$

Then it is inhomogenous linear AWP

$$
\boxed{ 
u^\prime (t) = Au + b(t), \, t \in [t_0, t_f], \qquad u(t_0) = u^0
}
$$

## 3.2. Solver

To simplify the problem, we will set $L=1$, $D=1$ and $v=1$. Also we will test the solver for the most simplest case when Nullrandbedingungen  $g_1(t) = g_2(t) = 0$ and costant Quelle $q(x,1) = 1$.

In [ ]:
def tridiag(low, mid, up):
    return np.diag(low, -1) + np.diag(mid, 0) + np.diag(up, 1)

In [ ]:
D = 1
L = 1.
v = 1
n = 5
hx = L/n

# Matrix A
diagonal = -4*D*np.ones(n-1)
upper = (2*D-v*hx)*np.ones(n-2)
lower = (2*D+v*hx)*np.ones(n-2)
matrix = tridiag(lower, diagonal, upper)
A = matrix /(2*(hx**2))
#print(A)

# Vector b
g_1 = 0
g_2 = 0

# Quelle
def q_1(j, t): 
    return 1

# b(t) for Konvektionsdiffusionsgleichung
def b_kd(t):
    b_1 = np.zeros(n-1)
    
    for i in range (n-1):
        b_1[i] = q_1(i+1, t)

    b_2 = np.zeros(n-1)
    b_2[0] = g_1*(2*D+v*hx)
    b_2[-1] = g_2*(2*D-v*hx)
    
    return b_1 + (1/(2*hx**2))*b_2

print("A = ", A)
eigenvalues, _ = eig(A)
eigenvalues.sort()
print("eigenvalues are ", eigenvalues)

Before solving with numerical solver such as implicit and explicit Euler Method, we should decide the Zeitschrittbeschranke. As we found in Aufgabe 1, all eigenvalues of matrix $A$ sould be all real and negative numbers. With eigenvalues, we can decide the value of Zeitschrittweit $ \tau $ to be used for the numerical solver so that process will be asymptotic stable.   

- Explicit Euler Method
$$ y^j = y^0 (1 + h\lambda)^j$$
To have aymptotic stability, $\lim_{j \to \infty} y^j$ should approach $0$ and it gives the restriction for $ 1+h\lambda$ 
$$ -1 < 1 + h\lambda < 1 $$
$$ -2 < h\lambda < 0 $$
$$  h < \frac{2}{|\lambda| } $$
Therefore the Zeitschrittbeschranke, the maximum Zeitschritt $\tau$, is :
$$ \tau < \frac{2}{| \lambda_\mathrm{max} |} $$


- Implicit Euler Method
$$ y^j = y^0 \left( \frac{1}{1 - h\lambda} \right)^j $$
Since eigenvalues of matrix $A$ are negative ($\lambda \ll 0$),  $\frac{1}{1 - h\lambda}$ is always between $(0,1)$, regardless of the value of $h$. Therefore, we don't need to set the limitation for the maximum Zeitschritt $\tau$. Regardless of $\tau = h$, the process will be always converged. 

The matrix $A$ for our case is 
$$ A =  \begin{pmatrix}
-50 & 22.5 & 0 & 0 \\
27.5 & -50 &22.5  &0  \\
0 & 27.5 & -50 & 22.5 \\
0 & 0 & 27.5 & -50 \\
\end{pmatrix}$$
The eigenvalues are $-90.248$, $-65.373$, $-34.627$ and $-9.752$. As we expected, they are all real and negative. For explicit Euler method, we should set the maximum Zeitschrittweit as 
$$ \tau < \frac{2}{| \lambda_\mathrm{max} |} = \frac{2}{| -90.248 |} \approx 0.0222$$
For implicit Euler method, we can use any value of Zeitschrittweit.

In [ ]:
# Function
def f(t, y): # y means u here
    return np.matmul(A,y) + b_kd(t)

def f_jac(t, y):
    return A

# Anfangsbedingung
u0 = np.zeros(n-1) # from u_1 to u_(n-1)
t0 = 0
tf = 1
t_span = (t0, tf)       # Zeitintervall als Tupel

In [ ]:
fig = plt.figure(figsize=(10,10))

N  = 10   # Azahl Zeitschritte
tk = np.linspace(t0, tf, N+1)
xj = np.linspace(0, L, n+1)
y_all_im = np.zeros( (n+1, N+1) )  # erster Index Ort, zweiter Index Zeit
y_all_ex = np.zeros( (n+1, N+1) )  # erster Index Ort, zweiter Index Zeit

ax1 = fig.add_subplot(221,projection='3d')
solution_ex = ode.euler_forward(f, t_span, u0, tk)
u_k_ex = solution_ex.y # from u_1 to u_(n-1)
# add the 'Randpunkte'
y_all_ex[0, :] = g_1
y_all_ex[-1, :] = g_2
# add the points, which are produced by the explicit euler method
for k in range(1,n):    
    y_all_ex[k, :] = u_k_ex[k-1, :]
TK, XJ = np.meshgrid(tk, xj)    # erzeugt Matrizen derselben Dimension wir y_all
ax1.plot_wireframe(TK, XJ, y_all_ex)
ax1.set_xlabel('t')
ax1.set_ylabel('x')
ax1.set_zlabel('u')
ax1.set_title('explicit Euler, h=0.1')

ax2 = fig.add_subplot(222,projection='3d')
solution_im = ode.euler_backward(f, t_span, u0, tk, f_jac)
u_k_im = solution_im.y # from u_1 to u_(n-1)
# add the 'Randpunkte'
y_all_im[0, :] = g_1
y_all_im[-1, :] = g_2
# add the points, which are produced by the implicit euler method 
for k in range(1,n):
    y_all_im[k, :] = u_k_im[k-1, :]
TK, XJ = np.meshgrid(tk, xj)    # erzeugt Matrizen derselben Dimension wir y_all
ax2.plot_wireframe(TK, XJ, y_all_im)
ax2.set_xlabel('t')
ax2.set_ylabel('x')
ax2.set_zlabel('u')
ax2.set_title('implicit Euler, h=0.1')



N  = 100   # Azahl Zeitschritte
tk = np.linspace(t0, tf, N+1)
xj = np.linspace(0, L, n+1)
y_all_im = np.zeros( (n+1, N+1) )  # erster Index Ort, zweiter Index Zeit
y_all_ex = np.zeros( (n+1, N+1) )  # erster Index Ort, zweiter Index Zeit

ax3 = fig.add_subplot(223,projection='3d')
solution_ex = ode.euler_forward(f, t_span, u0, tk)
u_k_ex = solution_ex.y # from u_1 to u_(n-1)
# add the 'Randpunkte'
y_all_ex[0, :] = g_1
y_all_ex[-1, :] = g_2
# add the points, which are produced by the explicit euler method
for k in range(1,n):    
    y_all_ex[k, :] = u_k_ex[k-1, :]
TK, XJ = np.meshgrid(tk, xj)    # erzeugt Matrizen derselben Dimension wir y_all
ax3.plot_wireframe(TK, XJ, y_all_ex)
ax3.set_xlabel('t')
ax3.set_ylabel('x')
ax3.set_zlabel('u')
ax3.set_title('explicit Euler, h=0.01')

ax4 = fig.add_subplot(224,projection='3d')
solution_im = ode.euler_backward(f, t_span, u0, tk, f_jac)
u_k_im = solution_im.y # from u_1 to u_(n-1)
# add the 'Randpunkte'
y_all_im[0, :] = g_1
y_all_im[-1, :] = g_2
# add the points, which are produced by the implicit euler method 
for k in range(1,n):
    y_all_im[k, :] = u_k_im[k-1, :]
TK, XJ = np.meshgrid(tk, xj)    # erzeugt Matrizen derselben Dimension wir y_all
ax4.plot_wireframe(TK, XJ, y_all_im)
ax4.set_xlabel('t')
ax4.set_ylabel('x')
ax4.set_zlabel('u')
ax4.set_title('implicit Euler, h=0.01')

plt.show()

From 2 graphs of implicit Euler method with $h=0.1$ and $h=0.01$, the numerical solution of both cases behave like continous solution. However, for expicit Euler method, only case with Euler $h=0.01$ behaves like continous solution. When $h=0.1$, the shape of graph is not converged, because Zeitshcrittweit $h=0.1$ is bigger that its Zeitschrittbeschranke $0.0222$.    